# TensorBoard test notebook
Here is the code to create a simple model visualisation with TensorBoard. This one uses the automatic logging features that are are included when using anything related to TensorFLow.

In [ ]:
# The requirements:

%pip install tensorboard tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation 
from keras.utils import np_utils 

In [3]:
# Separating the data into training and testing sets
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# All values are between 0 and 255, dividing them to have values from 0 to 1:
X_train_divided = X_train / 255
X_test_divided = X_test / 255

# Setting a few lists of variables to make varying the model output easy. 
first_layer_nodes = [100, 50, 100, 50, 100, 50]
first_layer_activation = ["relu", "sigmoid", "softmax", "relu", "sigmoid", "softmax"]
second_layer_nodes = [10, 15, 25, 25, 15, 10]
second_layer_activation = ["sigmoid", "softmax", "relu", "relu", "sigmoid", "softmax"]

In [4]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

Since TensorBoard creates a log file to visualise metrics and logged information, it's a good idea to clear previous log files, especially when running the same tests multiple times.
Uncomment if you need to clear previous logs (the folder name is defined at the start of the run logging, in the code cell after):

In [5]:
# !rm -rf ./TensorBoard_test_logs/

Remember to define the directory and name for each logged run, otherwise they will overwrite eachother. As you can see, I did it by iterating over a loop and using that to name my runs, but this can be done in other ways too, like manually or automatically using the "datetime" library. Personally, I find using datetime nice so here's a way to create the logging directory that way:

%pip install datetime \
import datetime \
log_dir = 'TensorBoard_test_logs/fit' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

Something else to consider is that in this case, the model is madde using keras (from TensorFlow, that also created TensorBoard), so automatic logging is available and used here.

In [6]:
for i in range(6):
    # Naming the runs and saving them in the same directory, so that we can upload the whole directory later.
    log_dir = "TensorBoard_test_logs/run_{}".format(i+1)
    tensorboard_callback= tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Model creation, compilation and training
    model = Sequential([
        keras.layers.Flatten(input_shape = (28,28), name = 'layers_flatten'),
        keras.layers.Dense(first_layer_nodes[i], first_layer_activation[i], name = 'layers_dense_1'),
        keras.layers.Dense(second_layer_nodes[i], second_layer_activation[i], name = 'layers_dense_2')
    ])
    model.compile(
        optimizer = 'adam',
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )
    model.fit(
        X_train_divided,
        y_train, 
        epochs = 5, 
        validation_data = (X_test_divided, y_test), 
        callbacks = [tensorboard_callback]
    )

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2703 - accuracy: 0.9238 - val_loss: 0.1450 - val_accuracy: 0.9573
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1223 - accuracy: 0.9645 - val_loss: 0.1116 - val_accuracy: 0.9678
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0863 - accuracy: 0.9740 - val_loss: 0.0928 - val_accuracy: 0.9735
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0657 - accuracy: 0.9802 - val_loss: 0.0909 - val_accuracy: 0.9719
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0515 - accuracy: 0.9842 - val_loss: 0.0821 - val_accuracy: 0.9750
Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5171 - accuracy: 0.8788 - val_loss: 0.2658 - val_accuracy: 0.9266
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2329 - accuracy: 0.9345 - val_loss: 0.2011 - val_accuracy: 0.9432

To visualise logged information, run the following cell, specifying the log directory. This will serve TensorBoard on localhost, the link to which should be provided in the ouput (something like http://localhost:6006). Remember that if you stop the cell, you are stopping the localhost connection and so won't be able to load any new graphs for visualisation that you haven't already opened.

It is also possible to serve the logs to the tensorboard.dev service, for sharing purposes. You can optionally specify some parameters (metadata), like in the commented out example the name of the uploaded logs, and a short description. To see the version of this test that I already uploaded to tensorboard.dev, go to https://tensorboard.dev/experiment/foEbSji9RVuQlhleSXGb1A/

In [7]:
!tensorboard --logdir TensorBoard_test_logs/

# !tensorboard dev upload \
# --logdir TensorBoard_test_logs/ \
# --name "Visualising 6 models with different CNN layers" \
# --description "TensorBoard default logs for 6 MNIST digit recognition models, with varying performance according to layer nodes and layer activation."


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.12.3 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In the case of TensorBoard, unique featrues include the automatic logging of tensors from the keras model's training layers.